# 模型训练/调参/预测

In [3]:
from constant import *
import numpy as np
from sklearn.metrics import roc_curve
from hyperopt import hp, tpe, fmin, Trials, STATUS_OK
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
from data import loader,exporter
import pandas as pd

In [4]:
def ks_stat(y_true, y_pred):
    """计算KS值的自定义评估函数"""
    fpr, tpr, _ = roc_curve(y_true, y_pred)  # 计算ROC曲线
    ks_value = np.max(np.abs(tpr - fpr))  # KS统计量
    return ks_value


def lgb_ks_eval(y_pred, dataset):
    """用于LightGBM的自定义KS评估函数"""
    y_true = dataset.get_label()
    ks_value = ks_stat(y_true, y_pred)
    # 返回 (名称, 计算的 KS 值, 是否越高越好)
    return 'ks', ks_value, True


# 自定义目标函数，用于贝叶斯优化
def objective(params, X, y, n_folds=5):
    """贝叶斯优化的目标函数，返回负的验证集AUC分数"""
    # 设置模型参数
    params['boosting_type'] = 'gbdt'  # 
    params['objective'] = 'binary'
    params['metric'] = 'auc'
    params['verbose'] = -1
    params['early_stopping_round'] = 20
    params['n_estimators'] = 10000  # 固定较大的迭代次数，使用早停机制

    # 交叉验证
    kf = StratifiedKFold(n_splits=n_folds, shuffle=True)
    ks_scores = []

    for train_idx, valid_idx in kf.split(X, y):
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

        # 建立LGBM模型
        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_valid = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

        # 训练模型
        model = lgb.train(params,
                          lgb_train,
                          valid_sets=[lgb_valid],
                          feval=lgb_ks_eval,
                          callbacks=[lgb.early_stopping(stopping_rounds=50),
                                     lgb.log_evaluation(10)]
                          )

        # 验证集预测概率
        y_pred = model.predict(X_valid, num_iteration=model.best_iteration)

        # 计算ks分数
        ks = ks_stat(y_valid, y_pred)
        ks_scores.append(ks)

    # 返回平均KS分数的负值作为最小化目标
    return {'loss': -np.mean(ks_scores), 'status': STATUS_OK}


# 超参数空间
param_space = {
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),  # 学习率
    'num_leaves': hp.choice('num_leaves', np.arange(20, 150, dtype=int)),  # 叶子数
    'max_depth': hp.choice('max_depth', np.arange(3, 12, dtype=int)),  # 树的最大深度
    'min_child_weight': hp.uniform('min_child_weight', 0.001, 10),  # 子叶节点的最小权重
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),  # 样本列采样率
    'scale_pos_weight': hp.uniform('scale_pos_weight', 1, 10),
    'subsample': hp.uniform('subsample', 0.5, 1.0),  # 样本采样率
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),  # L1正则化
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0)  # L2正则化
}


# 贝叶斯优化调参函数
def bayesian_optimize_lgbm(X, y, param_space, max_evals=50):
    trials = Trials()

    # 使用fmin函数进行贝叶斯优化
    best_params = fmin(
        fn=lambda params: objective(params, X, y),  # 优化目标
        space=param_space,  # 参数空间
        algo=tpe.suggest,  # 使用TPE算法
        max_evals=max_evals,  # 最大评估次数
        trials=trials  # 记录每次评估的结果
    )

    return best_params, trials

In [6]:
# 读取数据

df = pd.read_csv(f'{dir_preprocess}/v1.csv')
df.drop(columns=['CUST_NO'], inplace=True)

X = df[df['SRC'] == 'train']
X.drop(columns=['SRC'], inplace=True)
y = X.pop("FLAG")  # 目标标签列

# 执行贝叶斯优化
best_params, trials = bayesian_optimize_lgbm(X, y, param_space, max_evals=50)

print("Best parameters found:", best_params)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

C:\Users\zangq\AppData\Local\Temp\ipykernel_134712\2851928622.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(columns=['SRC'], inplace=True)
D:\workspace\misc\OneDay\venv\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")



Training until validation scores don't improve for 50 rounds
[10]	valid_0's auc: 0.726011	valid_0's ks: 0.345627   
[20]	valid_0's auc: 0.733065	valid_0's ks: 0.357893   
[30]	valid_0's auc: 0.734064	valid_0's ks: 0.359517   
[40]	valid_0's auc: 0.734908	valid_0's ks: 0.357798   
Training until validation scores don't improve for 50 rounds
  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

D:\workspace\misc\OneDay\venv\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")



[10]	valid_0's auc: 0.727855	valid_0's ks: 0.357708   
[20]	valid_0's auc: 0.735635	valid_0's ks: 0.351947   
[30]	valid_0's auc: 0.740199	valid_0's ks: 0.361179   
[40]	valid_0's auc: 0.741211	valid_0's ks: 0.361403   
[50]	valid_0's auc: 0.743439	valid_0's ks: 0.357493   
  0%|          | 0/50 [00:01<?, ?trial/s, best loss=?]

...

[10]	valid_0's auc: 0.74087	valid_0's ks: 0.370189                                 
[20]	valid_0's auc: 0.746862	valid_0's ks: 0.382569                                
[30]	valid_0's auc: 0.746322	valid_0's ks: 0.373788                                
 98%|█████████▊| 49/50 [02:57<00:03,  3.95s/trial, best loss: -0.40831624888276224]

D:\workspace\misc\OneDay\venv\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")



Training until validation scores don't improve for 50 rounds                       
[10]	valid_0's auc: 0.732156	valid_0's ks: 0.345387                                
[20]	valid_0's auc: 0.745038	valid_0's ks: 0.361438                                
[30]	valid_0's auc: 0.746668	valid_0's ks: 0.366405                                
[40]	valid_0's auc: 0.748112	valid_0's ks: 0.370027                                
[50]	valid_0's auc: 0.746142	valid_0's ks: 0.375617                                
 98%|█████████▊| 49/50 [02:58<00:03,  3.95s/trial, best loss: -0.40831624888276224]

D:\workspace\misc\OneDay\venv\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")



Training until validation scores don't improve for 50 rounds                       
[10]	valid_0's auc: 0.747888	valid_0's ks: 0.378478                                
[20]	valid_0's auc: 0.754798	valid_0's ks: 0.388624                                
[30]	valid_0's auc: 0.756477	valid_0's ks: 0.394202                                
100%|██████████| 50/50 [02:59<00:00,  3.59s/trial, best loss: -0.40831624888276224]
Best parameters found: {'colsample_bytree': 0.6553725084346377, 'learning_rate': 0.15024653076580058, 'max_depth': 0, 'min_child_weight': 8.633337189221727, 'num_leaves': 33, 'reg_alpha': 0.14994023674796636, 'reg_lambda': 0.6444882318414292, 'scale_pos_weight': 1.0025584197567623, 'subsample': 0.6944630864997181}


In [11]:
# 5折交叉验证
def lgb_dart_5_fold(X, y, best_params, n_folds=5):
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True)
    fold_scores = []
    final_predictions = np.zeros(len(X))  # 用于存储每一折的预测结果加总

    for fold, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

        # 训练模型
        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_valid = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

        # 训练模型
        model = lgb.train(best_params,
                          lgb_train,
                          valid_sets=[lgb_valid],
                          feval=lgb_ks_eval,
                          callbacks=[lgb.early_stopping(stopping_rounds=50),
                                     lgb.log_evaluation(10)]
                          )

        # 保存模型
        model_path = f"{dir_model}/lgbm_fold_{fold + 1}.bin"
        model.save_model(model_path)
        print(f"Model for fold {fold + 1} saved at {model_path}")

        # 验证集预测
        y_pred = model.predict(X_valid, num_iteration=model.best_iteration)

        # 计算KS和最大F2分数
        score = ks_stat(y_valid, y_pred)
        fold_scores.append(score)

        # 将每一折的预测结果加总
        final_predictions[valid_idx] = y_pred

        print(f"Fold {fold + 1} - Score: {score:.4f}")

    # 最终综合预测结果的评分
    overall_score = ks_stat(y, final_predictions)
    # overall_score, overall_best_threshold = calculate_best_f2(y, final_predictions)

    print(f"\nOverall Score: {overall_score:.4f}")

    return overall_score

In [12]:
# 执行5折交叉验证
best_score = lgb_dart_5_fold(X, y, best_params)
print(f"Best score: {best_score}")


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005081 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6919
[LightGBM] [Info] Number of data points in the train set: 42475, number of used features: 87
[LightGBM] [Info] Start training from score 0.043861
Training until validation scores don't improve for 50 rounds
[10]	valid_0's l2: 0.0398541	valid_0's ks: 0.35016
[20]	valid_0's l2: 0.0394434	valid_0's ks: 0.348851
[30]	valid_0's l2: 0.0393481	valid_0's ks: 0.36822
[40]	valid_0's l2: 0.0392567	valid_0's ks: 0.383873
[50]	valid_0's l2: 0.039364	valid_0's ks: 0.372055
[60]	valid_0's l2: 0.0394247	valid_0's ks: 0.369607
[70]	valid_0's l2: 0.0394121	valid_0's ks: 0.36224
[80]	valid_0's l2: 0.0393955	valid_0's ks: 0.361911
Early stopping, best iteration is:
[39]	valid_0's l2: 0.0392539	valid_0's ks: 0.378896
Model for fold 1 saved at /data/contest/2024\2\model/lgbm_fold_1.bin
Fold 1 - Score: 0.3789
[Li

In [26]:
# 测试集预测并生成结果文件
def predict_test_set(test_data, n_folds=5, output_file='upload.csv'):
    test_predictions = np.zeros(len(test_data))

    # 依次加载每一折的模型进行预测
    for fold in range(n_folds):
        model_path = f"{dir_model}/lgbm_fold_{fold + 1}.bin"
        model = lgb.Booster(model_file=model_path)

        # 对测试集进行预测，并将每一折的预测加总
        test_predictions += model.predict(test_data)

    # 平均每一折的预测
    test_predictions /= n_folds

    # 生成预测文件
    submission = pd.DataFrame({'CUST_NO': test_data.index, 'PRED': test_predictions})
    submission.to_csv(output_file, index=False)
    print(f"Test predictions saved to {output_file}")